### Basic Imports

In [27]:
import nbimporter
from typing import Dict, Tuple, List
import numpy as np
from tqdm import tqdm

### Env Vars

In [28]:
QUBITS_NUM = 4 
N = 16
K = 4
NUM_SHOTS = 1024
NUM_ITERATIONS = 10

w_vector = np.asarray([4,3,2,1])

### Simulator Backend

In [29]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

simulator_backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [30]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=NUM_ITERATIONS)

### K input states (computational basis)

In [31]:
from utiles import *

In [32]:
input_states = get_first_k_eigenvectors_from_n_computational_basis(K, N)

### Ansatz State

In [33]:
from ansatz_circuit_item2 import get_full_variational_quantum_circuit

In [34]:
init_circuit_params = {
    "thetas": np.random.uniform(low=0, high=2*np.pi, size=8),
    "phis": np.random.uniform(low=0, high=2*np.pi, size=4),
    "D1": 2,
    "D2": 6
}

In [35]:
def prepare_circuit_params(thetas) -> Dict:
     return {
    "thetas": thetas[4:],
    "phis": thetas[:4],
    "D1": 2,
    "D2": 6
     }

In [36]:
def get_ansatz_state(circuit_params, input_state):
    circuit_params_with_input_state = {**circuit_params, "input_state": input_state}
    return get_full_variational_quantum_circuit(**circuit_params_with_input_state)

## Expectation Value

### convert hamiltonian to pauli strings

In [37]:
#TODO: change the a_i and J_ij params into random values
from qiskit.opflow import X, Z, Y, I, H, S
from qiskit.opflow import ListOp

H_transverse_ising = 0.5*((I^I^I^X) + (I^I^X^I) + (I^X^I^I) + (X^I^I^I) + \
                          (Z^Z^I^I) + (Z^I^Z^I) + (Z^I^I^Z) + (I^Z^Z^I) + \
                          (I^Z^I^Z) + (I^I^Z^Z))

In [38]:
def transfrom_hamiltonian_into_pauli_string(hamiltonian) -> List:
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_strings

### pauli string reduction to sigma_z's

In [39]:
from qiskit.circuit.library.standard_gates import HGate, SGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [40]:
reducing_to_pauli_z_mapping = {
    'I': 'I',
    'Z': 'Z',
    'X': 'Z',
    'Y': 'Z'
} 

In [41]:
def reduce_pauli_matrixes_into_sigma_z(pauli_string) -> str:
    reduced_pauli_string = ""
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = str(pauli_string[matrix_index])
        reduced_pauli_matrix = reducing_to_pauli_z_mapping[pauli_matrix]
        reduced_pauli_string = reduced_pauli_matrix + reduced_pauli_string
    
    return reduced_pauli_string

In [42]:
def add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string, quantum_circuit):
    quantum_registers = QuantumRegister(QUBITS_NUM, name="qubit")
    additional_circuit_layer = QuantumCircuit(quantum_registers)
    
    for quantum_register_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
        if pauli_string == "Y":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
            additional_circuit_layer.append(SGate(), [quantum_registers[quantum_register_index]])
                
    extended_quantum_circuit = quantum_circuit.compose(additional_circuit_layer)
    return extended_quantum_circuit

### probabilities distribution

In [43]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [44]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def convert_pauli_string_into_str(pauli_string) -> str:
    return str(pauli_string)

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, ansatz_circuit):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    pauli_string_str = convert_pauli_string_into_str(pauli_string)
    extended_ansatz_circuit = add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string_str, ansatz_circuit)
    probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(extended_ansatz_circuit)
    
    reduced_pauli_string = reduce_pauli_matrixes_into_sigma_z(pauli_string)
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

In [45]:
def get_expectation_value(ansatz_circuit, pauli_strings):
    total_expection_value = 0
    
    for pauli_string in pauli_strings:
        total_expection_value += calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                    pauli_string, ansatz_circuit)
    
    return total_expection_value

## Objective Function

In [46]:
from qiskit import assemble, transpile

def cost_function(thetas):
    L_w = 0
    circuit_params = prepare_circuit_params(thetas)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
    
    pauli_strings = transfrom_hamiltonian_into_pauli_string(H_transverse_ising)
    
    for j in tqdm(range(K)):
        ansatz_state = get_ansatz_state(circuit_params, computational_eigenvectors[j])
        print(w_vector[j])
        L_w += w_vector[j]*get_expectation_value(ansatz_state, pauli_strings)
        
    return L_w

### Optimization

In [47]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
optimizer_result = bfgs_optimizer.minimize(cost_function,x0=initial_thetas)
print(optimizer_result)
optimal_thetas = prepare_circuit_params(optimizer_result.x)

computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
optimal_ansatz_state = get_ansatz_state(optimal_thetas, computational_eigenvectors[K-1])
pauli_strings = transfrom_hamiltonian_into_pauli_string(H_transverse_ising)
print(get_expectation_value(optimal_ansatz_state, pauli_strings))

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

4


 25%|█████████████████████                                                               | 1/4 [00:00<00:01,  1.53it/s]

3


 50%|██████████████████████████████████████████                                          | 2/4 [00:01<00:01,  1.53it/s]

...

2


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:02<00:00,  1.01it/s]

1


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.01s/it]


{   'fun': 40.31640625,
    'jac': None,
    'nfev': 299,
    'nit': 23,
    'njev': 23,
    'x': array([3.76403322, 1.71482327, 3.09994492, 1.91250943, 2.75088221,
       2.38749267, 5.21406125, 2.48253078, 5.59435832, 3.8987431 ,
       6.22117064, 4.57036918])}
3.79296875


In [56]:
optimal_thetas = prepare_circuit_params(np.asarray([3.76403322, 1.71482327, 3.09994492, 1.91250943, 2.75088221,
       2.38749267, 5.21406125, 2.48253078, 5.59435832, 3.8987431 ,
       6.22117064, 4.57036918]))

pauli_strings = transfrom_hamiltonian_into_pauli_string(H_transverse_ising)
computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
for eigenvalue_index, eigenvector in enumerate(computational_eigenvectors):
    print(eigenvector)
    optimal_ansatz_state = get_ansatz_state(optimal_thetas, eigenvector)
    print("eigenvalue for k =", eigenvalue_index)
    print(get_expectation_value(optimal_ansatz_state, pauli_strings))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 0
4.244140625
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 1
4.01953125
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 2
3.951171875
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 3
3.802734375


In [55]:
from numpy import linalg as LA
print(LA.eig(H_transverse_ising.to_matrix()))

(array([-2.37228132e+00+0.00000000e+00j, -3.02775638e-01+0.00000000e+00j,
        1.00000000e+00+0.00000000e+00j,  3.37228132e+00+0.00000000e+00j,
        3.30277564e+00+0.00000000e+00j,  6.18033989e-01+0.00000000e+00j,
       -1.00000000e+00+0.00000000e+00j, -1.61803399e+00+0.00000000e+00j,
       -1.61803399e+00+0.00000000e+00j, -1.00000000e+00+0.00000000e+00j,
       -1.61803399e+00+0.00000000e+00j,  6.18033989e-01+0.00000000e+00j,
        6.18033989e-01+0.00000000e+00j, -1.40724346e-16+3.38963670e-32j,
        1.26249568e-16+7.70371978e-34j, -3.42283876e-17-6.14669432e-33j]), array([[ 8.11964088e-02+0.00000000e+00j, -2.04908337e-01+0.00000000e+00j,
        -2.50000000e-01+0.00000000e+00j,  6.56435178e-01-0.00000000e+00j,
         6.76766262e-01+0.00000000e+00j,  3.90189627e-17-0.00000000e+00j,
         1.04565593e-17-0.00000000e+00j,  4.48315450e-18+0.00000000e+00j,
        -8.76732475e-19+0.00000000e+00j, -5.64406524e-18+0.00000000e+00j,
         5.06628589e-18+0.00000000e+00j, -4